In [1]:
from time import sleep, perf_counter
import concurrent.futures
from bs4 import BeautifulSoup 
import requests
import json
import pandas as pd

In [2]:
base_url = "https://www.olympedia.org"
initial_url = "https://www.olympedia.org/countries"
country_noc = []
countries_urls = []
response = requests.get(initial_url)
if response.status_code == 200:
    page = BeautifulSoup(response.content, "lxml")
    table_countries = page.find('tbody').find_all('tr')
    for countries in table_countries:
        if countries.find(attrs = {'class': 'glyphicon glyphicon-ok'}):
            country_url = base_url + countries.find_all('a')[1]['href']
            countries_urls.append(country_url)
            country_noc_data = {
                "noc": countries.find_all("a")[0].text,
                "country": countries.find_all("a")[1].text
            }
            country_noc.append(country_noc_data)
    print('Country URLs successfully fetched')
    print('Number of participated countries:', len(countries_urls))
    print('Number of countries_noc:', len(country_noc))
        
else:
    print(response.raise_for_status())

Country URLs successfully fetched
Number of participated countries: 234
Number of countries_noc: 234


In [3]:
def get_events_url(country_url):
    events_urls = []
    response = session_event.get(country_url)
    if response.status_code == 200:
        country_page = BeautifulSoup(response.content, "lxml")
        if country_page.find('tbody') is not None:
            games_table = country_page.find('tbody').find_all('tr')
            for games in games_table:
                events_url = base_url + games.find_all('a')[1]['href']
                events_urls.append(events_url)
    else:
        print(response.raise_for_status())

    return events_urls

In [4]:
start = perf_counter()
with requests.Session() as session_event:
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        events_urls = list(executor.map(get_events_url, countries_urls))
end = perf_counter()
print(f"Elapsed Time: {end-start:.02f} seconds")


Elapsed Time: 56.10 seconds


In [5]:
events_url_falttened = [event_url for country_events_urls in events_urls for event_url in country_events_urls]

In [8]:
def get_athletes_url(events_url):
    athletes_urls = []
    response = session_event.get(events_url)
    if response.status_code == 200:
        event_page = BeautifulSoup(response.content, "lxml")
        if event_page.find('tbody') is not None:
            athletes_table = event_page.find('tbody').find_all('a')
            for all_url in athletes_table:
                if 'athlete' in all_url['href']:
                    athletes_url = base_url + all_url['href']
                    athletes_urls.append(athletes_url)
    else:
        print(response.raise_for_status())
    sleep(1)
    return athletes_urls
    

In [10]:
start = perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=75) as executor:
    athletes_urls = list(executor.map(get_athletes_url, events_url_falttened))
end = perf_counter()
print(f"Elapsed Time: {end-start:.02f} seconds")

HTTPError: 504 Server Error: Gateway Time-out for url: https://www.olympedia.org/countries/FRA/editions/2

In [8]:
athletes_url_falttened = [athlete_url for events_athletes_urls in athletes_urls for athlete_url in events_athletes_urls]

In [9]:
unique_athletes_urls = list(set(athletes_url_falttened))
print('Number of unique athletes: ', len(unique_athletes_urls))

In [10]:
with open('data/athletes_urls.json', 'w') as file:
   json.dump(unique_athletes_urls, file)

In [11]:
with open('data/events_urls.json', 'w') as file:
   json.dump(events_url_falttened, file)

In [12]:
with open('data/countries_urls.json', 'w') as file:
   json.dump(countries_urls, file)

In [9]:
country_noc_df = pd.DataFrame(country_noc)
country_noc_df.to_csv("data/countries_noc.csv", index = False)